# Инфраструктура для моделей машинного обучения. Практическая работа

# Цель практической работы

Потренироваться в использовании библиотек PySpark SQL и PySpark ML для предобработки данных и обучения моделей.

# Что входит в практическую работу

1. Инициализация спарк-сессии.
2. Загрузка данных.
3. Ознакомление с данными.
4. Преобразование типов столбцов.
5. Очистка данных.
6. Feature-инжиниринг.
7. Векторизация фичей.
8. Создание и обучение модели.
9. Выбор лучшей модели.
10. Обратная связь.


# Что оценивается

- Пройдены все этапы работы.
- Спарк-сессия успешно запущена.
- Данные прочитаны.
- Все колонки с числовыми значениями преобразованы в числовые типы данных (Int или Double).
- Отфильтрованы все строки с Null-значениями.
- Созданы новые фичи.
- Все категориальные колонки преобразованы в числовой вид, выполнены все этапы векторизации признаков.
- Выборка разделена на обучающую и тестовую.
- Создано три объекта: модель, сетка гиперпараметров и evaluator.
- Создан объект класса CrossValidator и обучен на обучающей выборке.
- Выбрана лучшая модель, посчитана метрика качества лучшей модели.


# Задача

Используя данные о клиентах телекоммуникационной компании, обучите модель, предсказывающую их отток.

Описание данных, с которыми вы будете работать:

* **CustomerID**: ID клиента.
* **Gender**: пол клиента.
* **SeniorCitizen**: пенсионер ли клиент (1 — да, 0 — нет).
* **Partner**: есть у клиента партнёр (жена, муж) или нет (Yes/No).
* **Dependents**: есть ли у клиента инждивенцы, например дети (Yes/No).
* **Tenure**: как много месяцев клиент оставался в компании.
* **PhoneService**: подключена ли у клиента телефонная служба (Yes/No).
* **MultipleLines**: подключено ли несколько телефонных линий (Yes, No, No phone service).
* **InternetService**: интернет-провайдер клиента (DSL, Fiber optic, No).
* **OnlineSecurity**: подключена ли у клиента услуга онлайн-безопасности (Yes, No, No internet service)
* **OnlineBackup**: подключена ли услуга резервного копирования онлайн (Yes, No, No internet service).
* **DeviceProtection**: подключена ли услуга защиты устройства (Yes, No, No internet service)
* **TechSupport**: есть ли у клиента техническая поддержка (Yes, No, No internet service).
* **StreamingTV**: подключена ли услуга потокового телевидения (Yes, No, No internet service).
* **StreamingMovies**: подключена ли услуга стримингового воспроизведения фильмов (Yes, No, No internet service).
* **Contract**: тип контракта клиента (Month-to-month, One year, Two year).
* **PaperlessBilling**: есть ли безбумажный счёт.
* **PaymentMethod**: способ оплаты услуг (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)).
* **MonthlyCharges**: сумма, которая списывается ежемесячно.
* **TotalCharges**: сумма, списанная за всё время.
* **Churn**: ушёл ли клиент (Yes/No). Это целевая переменная, которую нужно предсказать.


# 1. Инициализация спарк-сессии

Инициализируйте спарк-сессию.

Эта ячейка нужна для того, чтобы заргузить необходимые библиотеки и настроить окружение Google Colab для работы со Spark.

Просто запустите её перед выполением задания :)

In [1]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

In [3]:
from pyspark.sql import SparkSession



spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()


# 2. Загрузка данных
Загрузите данные, сохраните их в переменную типа sparkDataframe, используя метод read.csv (не забывайте про header и delimiter).

In [4]:
from google.colab import drive
drive.mount("/content/drive")

churn = spark.read.option("header",True).option("delimiter",",").csv("/content/drive/MyDrive/Colab Notebooks/WA_Fn-UseC_-Telco-Customer-Churn.csv")


Mounted at /content/drive


# 3. Ознакомление с данными
1. Выведите на экран первые несколько строк датафрейма.


In [5]:
churn.show(7)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    


2. Выведите общее количество строк датафрейма.



In [6]:
churn.count()

7043

3. Выведите структуру (схему) датафрейма.

In [7]:
churn.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



# 4. Преобразование типов столбцов
Преобразуйте тип столбцов у числовых признаков (Int — если признак целочисленный, Double — если признак не целочисленный). Сохраните преобразованный датафрейм в новую переменную.

## Совет

Если вам сложно выполнить это задание, изучите дополнительные материалы: [об операторе Cast](https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/), [об операторе Select](https://sparkbyexamples.com/pyspark/select-columns-from-pyspark-dataframe/).



In [46]:
from pyspark.sql.functions import col,sum,avg,max,count,expr

churn_formatted = churn.select(
    col("gender").cast("string"),
    col("SeniorCitizen").cast("Int"),
    col("Partner").cast("string"),
    col("Dependents").cast("string"),
    col("tenure").cast("Int"),
    col("PhoneService").cast("string"),
    col("MultipleLines").cast("string"),
    col("InternetService").cast("string"),
    col("OnlineSecurity").cast("string"),
    col("OnlineBackup").cast("string"),
    col("DeviceProtection").cast("string"),
    col("TechSupport").cast("string"),
    col("StreamingTV").cast("string"),
    col("StreamingMovies").cast("string"),
    col("Contract").cast("string"),
    col("PaperlessBilling").cast("string"),
    col("PaymentMethod").cast("string"),
    col("MonthlyCharges").cast("Double"),
    col("TotalCharges").cast("Double"),
    col("Churn").cast("string")
    )

# 5. Очистка данных
Проверьте, есть ли в какой-либо колонке Null-значения. Для этого можно использовать your_dataframe.filter(col("colname")).isNull()).

Выведите на экран несколько строк с Null-значениями в одной из колонок.

Сохраните очищенный от строк с Null-значениями датафрейм в новую переменную. Для фильтрации этих значений можно использовать метод .isNotNull().

Колонок в датафрейме много, проверять каждую неудобно и долго. Подумайте, как упроситить эту работу, если использовать, например, перебор с циклом for.

[Примеры использования операторов isNull() и isNotNull()](https://sparkbyexamples.com/pyspark/pyspark-isnull/).


In [47]:
churn_formatted.columns

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [48]:
churn_formatted.filter(col('TotalCharges').isNull()).show(5)

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|       Yes|     0|          No|No phone service|            DSL|                Ye

In [49]:
for column in churn_formatted.columns:
  churn_formatted = churn_formatted.filter(col(column).isNotNull())

In [50]:
churn_formatted.count()

7032

In [51]:
churn_formatted.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|

# 6. Feature-инжиниринг
Добавьте в датафрейм одну или несколько новых фичей. Удалите колонки, которые, как вам кажется, нужно убрать из фичей. Обоснуйте свои решения.

In [52]:
from pyspark.sql.functions import when

def payments_have_grown_f(tenure, total, montly):
  return  montly > (total / tenure)


churn_formatted = churn_formatted.withColumn("payments_have_grown",payments_have_grown_f(col("tenure"),col("TotalCharges"),col("MonthlyCharges")))

churn_formatted = churn_formatted.select(
    col("gender").cast("string"),
    col("SeniorCitizen").cast("Int"),
    col("Partner").cast("string"),
    col("Dependents").cast("string"),
    col("tenure").cast("Int"),
    col("PhoneService").cast("string"),
    col("MultipleLines").cast("string"),
    col("InternetService").cast("string"),
    col("OnlineSecurity").cast("string"),
    col("OnlineBackup").cast("string"),
    col("DeviceProtection").cast("string"),
    col("TechSupport").cast("string"),
    col("StreamingTV").cast("string"),
    col("StreamingMovies").cast("string"),
    col("Contract").cast("string"),
    col("PaperlessBilling").cast("string"),
    col("PaymentMethod").cast("string"),
    col("MonthlyCharges").cast("Double"),
    col("TotalCharges").cast("Double"),
    col("Churn").cast("string"),
    col("payments_have_grown").cast("string")
    )


предположим, что наша организация успешно предлагает дополнительные услуги своим клиентам, для этого сравним размер текущего ежемесячного платежа с расчетным размером ежемесячного платежа при условии, что он оставался на протяжении всего времени пользования услугами компании неизменным - все просто, если текущий платеж больше расчетного - платеж вырос, гиппотеза подтверждена, если же нет - значит по мере пользования нашими услугами клиент постепенно отказывался от их части и, возможно, скоро совсем от нас уйдет.

Важно понимать, что в целях простоты эксперимента мы не учитываем постепенное повышение тарифов как таковое, но, думаю, его можно было бы заложить приблизительным процентом даже при отсутствии конкретных данных и посмотреть при каком проценте эта фича начала бы давать плюс к метрике на выходе из пайпа


#7. Векторизация фичей
Подготовьте данные к обучению:





1. Преобразуйте текстовые колонки в числа, используя StringIndexer.
Удалите столбцы со старыми (непреобразованными) признаками. Выведите на экран структуру получившегося датафрейма. Не забывайте о столбце Churn. Хоть он и выступает в задаче как таргет, он имеет текстовый тип, поэтому тоже должен быть закодирован числовыми значениями.

Чтобы использовать StringIndexer для всех категориальных признаков сразу, а не для каждого отдельно, можно применить сущность pipeline.

**Пример кода:**

##### #Задаём список текстовых колонок:
text_columns = ["text_col_1", "text_col_2", "text_col_3"]

##### #Задаём список StringIndexer'ов — сущностей, каждая из которых будет кодировать одну текстовую колонку числами. Имена преобразованных колонок будут заканчиваться на _index:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(<ваш датасет>) for column in text_columns]

##### #Создаём Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

##### #Скармливаем нашему pipeline датафрейм, удаляя старые колонки:
new_dataframe = pipeline.fit(<ваш датасет>).transform(<ваш датасет>).drop(*text_columns)


In [53]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#список колонок с текстовым типом
text_cols = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "Churn", "payments_have_grown"]


### Ваш код здесь ###
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(churn_formatted) for column in text_cols]
pipeline = Pipeline(stages=indexers)

churn_formatted_LE = pipeline.fit(churn_formatted).transform(churn_formatted).drop(*text_cols)

In [54]:
churn_formatted_LE.show(5)

+-------------+------+--------------+------------+------------+-------------+----------------+------------------+-------------------+---------------------+--------------------+------------------+----------------------+-----------------+-----------------+---------------------+--------------+----------------------+-------------------+-----------+-------------------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|gender_index|Partner_index|Dependents_index|PhoneService_index|MultipleLines_index|InternetService_index|OnlineSecurity_index|OnlineBackup_index|DeviceProtection_index|TechSupport_index|StreamingTV_index|StreamingMovies_index|Contract_index|PaperlessBilling_index|PaymentMethod_index|Churn_index|payments_have_grown_index|
+-------------+------+--------------+------------+------------+-------------+----------------+------------------+-------------------+---------------------+--------------------+------------------+----------------------+-----------------+-----------------+

2. Векторизуйте категориальные признаки, используя OneHotEncoder.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран структуру получившегося после преобразований датафрейма.


In [55]:
from pyspark.ml.feature import OneHotEncoder

#список категориальных колонок
features_inp  = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "payments_have_grown"]


### Ваш код здесь ###



features_inp_ind = [column+"_index" for column in features_inp]
features_out  = [column+"_vec" for column in features_inp_ind]

#for column in features_inp_ind:
#encoder = OneHotEncoder(inputCols = column, outputCols = column+'_vec')#.fit(churn_formatted_LE)
encoder = OneHotEncoder(inputCols = features_inp_ind, outputCols = features_out)#.fit(churn_formatted_LE)

fitted_encoder = encoder.fit(churn_formatted_LE)
churn_formatted_LE_vec = fitted_encoder.transform(churn_formatted_LE)#.drop(features_inp_ind)

In [56]:
for col in features_inp_ind:
  churn_formatted_LE_vec = churn_formatted_LE_vec.drop(col)

In [57]:
churn_formatted_LE_vec.show(5)

+-------------+------+--------------+------------+-----------+----------------+-----------------+--------------------+----------------------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+------------------+--------------------------+-----------------------+-----------------------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|Churn_index|gender_index_vec|Partner_index_vec|Dependents_index_vec|PhoneService_index_vec|MultipleLines_index_vec|InternetService_index_vec|OnlineSecurity_index_vec|OnlineBackup_index_vec|DeviceProtection_index_vec|TechSupport_index_vec|StreamingTV_index_vec|StreamingMovies_index_vec|Contract_index_vec|PaperlessBilling_index_vec|PaymentMethod_index_vec|payments_have_grown_index_vec|
+-------------+------+--------------+------------+-----------+----------------+-----------------+--------------------+--------

3. Объедините колонки фичей в один вектор, используя VectorAssembler.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран первые несколько строк и структуру получившегося датафрейма.

In [58]:
from pyspark.ml.feature import VectorAssembler

features = list(churn_formatted_LE_vec.drop("Churn_index").columns)
target = "Churn_index"

vectorizer = VectorAssembler(inputCols = features, outputCol = "features_vec")
#Создаём переменную vectorizer — объект класса VectorAssembler, цель которого — превратить колонки с фичами в векторы

churn_vectorised = vectorizer.transform(churn_formatted_LE_vec)
#Трансформируем при помощи vectorizer наш датафрейм и сохраним его в переменную realty_vectorised


In [60]:
for col in features:
  churn_vectorised = churn_vectorised.drop(col)
churn_vectorised.show(5)

+-----------+--------------------+
|Churn_index|        features_vec|
+-----------+--------------------+
|        0.0|(31,[1,2,3,6,11,1...|
|        0.0|(31,[1,2,3,4,5,6,...|
|        1.0|(31,[1,2,3,4,5,6,...|
|        0.0|(31,[1,2,3,4,5,6,...|
|        1.0|(31,[1,2,3,5,6,7,...|
+-----------+--------------------+
only showing top 5 rows



#8. Создание и обучение модели

1. Создайте модель — логистическую регрессию (используя LogisticRegression). В качестве параметров класса LogisticRegression укажите колонку фичей (параметр featuresCol), колонку-таргет (параметр labelCol) из датафрейма и имя колонки, в которую будут записываться предсказания (параметр predictionCol).

In [62]:
from pyspark.ml.classification import LogisticRegression

### Ваш код здесь ###
model = LogisticRegression(featuresCol = 'features_vec', labelCol = target)






2. Разделите датафрейм на обучающую и тестовую выборку.

In [63]:

### Ваш код здесь ###
churn_train, churn_test = churn_vectorised.randomSplit([0.8, 0.2])

3. Создайте объекты — сетки гиперпараметров для каждой модели, используя ParamGridBuilder. Так же, как и в ноутбуке из последнего видео, в сетку гиперпараметров можно добавить значения параметров regParam и elasticNetParam.

Вы можете ознакомиться [с документацией объекта LogisticRegression в PySpark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html) и добавить в сетку больше параметров.


In [64]:
from pyspark.ml.tuning import ParamGridBuilder

### Ваш код здесь ###
grid = ParamGridBuilder().addGrid(model.regParam, [0.5, 5]).addGrid(model.elasticNetParam, [0.01, 0.1]).build()


4. Создайте объект evaluator, который будет отвечать за метрику качества при обучении. Для этого используйте класс BinaryClassificationEvaluator со следующими параметрами: rawPredictionCol — колонка с предсказаниями, labelCol — колонка с таргетом.

У вас, возможно, возник вопрос, какую метрику качества берёт по умолчанию BinaryClassificationEvaluator. По умолчанию BinaryClassificationEvaluator будет рассчитывать areaUnderROC. Это метрика оценки площади под кривой ROC (Receiver Operating Characteristic), которая служит графической интерпретацией производительности модели. Эта метрика качества находится в пределах от 0 до 1. Чем выше метрика, тем более качественные предсказания делает модель.

In [66]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Ваш код здесь ###
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol = target)

5. Создайте объект CrossValidator, в качестве параметров укажите уже созданные вами модель, сетку гиперпараметров и evaluator.

6. Запустите обучение модели на тренировочной выборке. Сохраните обученную модель в новую переменную.

In [68]:
from pyspark.ml.tuning import CrossValidator

### Ваш код здесь ###
cv = CrossValidator(estimator = model, estimatorParamMaps = grid, evaluator = eval)
cv_model = cv.fit(churn_train)

#9. Выбор лучшей модели

1. Выберите лучшую модель, сохраните её в отдельную переменную, отобразите её параметры.

Вывод параметров модели в PySpark можно сделать, используя метод extractParamMap().

In [69]:

### Ваш код здесь ###
best_model = cv_model.bestModel
best_model.extractParamMap()

{Param(parent='LogisticRegression_263b76848ca0', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_263b76848ca0', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.01,
 Param(parent='LogisticRegression_263b76848ca0', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_263b76848ca0', name='featuresCol', doc='features column name.'): 'features_vec',
 Param(parent='LogisticRegression_263b76848ca0', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_263b76848ca0', name='labelCol', doc='label column name.'): 'Churn_index',
 Param(parent='LogisticRegression_263b76848ca0', name='maxBlockSizeInMB', doc='maximum memory 

2. Запустите лучшую модель в режиме предсказания на тестовой выборке. Сохраните предсказания в отдельную переменную. Выведите первые несколько строк датафрейма с предсказаниями на экран.

Запуск модели в режиме предсказания выполняется при помощи метода .transform(<тестовая выборка>).

In [75]:

### Ваш код здесь ###
churn_test_with_preds = best_model.transform(churn_test)
churn_test_with_preds.show(5)

+-----------+--------------------+--------------------+--------------------+----------+
|Churn_index|        features_vec|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+----------+
|        0.0|(31,[0,1,2,3,4,5,...|[0.20825132689640...|[0.55187548643003...|       0.0|
|        0.0|(31,[0,1,2,3,4,5,...|[0.26290617878597...|[0.56535056024564...|       0.0|
|        0.0|(31,[0,1,2,3,4,5,...|[0.63992775069127...|[0.65473712836875...|       0.0|
|        0.0|(31,[0,1,2,3,4,5,...|[1.04487322860718...|[0.73978920377417...|       0.0|
|        0.0|(31,[0,1,2,3,4,5,...|[1.97544923316402...|[0.87819521114000...|       0.0|
+-----------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [76]:
churn_train_with_preds = best_model.transform(churn_train)
eval.evaluate(churn_train_with_preds)

0.5815940590022091

3. Получите метрику качества модели. Для этого примените к объекту evaluator метод .evaluate(<ваш датафрейм с предсказаниями>).



In [77]:

### Ваш код здесь ###
eval.evaluate(churn_test_with_preds)

0.572033722060758

#10. Обратная связь
Вы ознакомились с возможностями двух мощных библиотек: PySpark SQL для предобработки данных и PySpark ML для машинного обучения.

Поделитесь впечатлениями от работы с новыми библиотеками. В чём они более удобны, чем уже знакомые вам Pandas и Sklearn, а в чём нет.

Сложно сказать, насколько спарковские варианты Ск-лерн удобнее или наоборот, чем "дефолтные" - опять же важно понимать, что с дефолтными мы уже как то общаемся (и, хочется думать, более-менее плотно) уже какое то продолжительное время, а тут - как будто кривое зеркало - все чуть по другому.

SQL все еще мы как в основном курсе не трогали особо, так и тут. сложно сравнивать.

Отдельно хочу выразить полушуточное "фи" по поводу необходимости каждый раз писать .show(x) для того, чтобы просто посмотреть на данные:)





# Как отправить работу на проверку

Загрузите файл с заданиями, откройте его через Jupyter Notebook в Google Colab. Скачайте файл с датасетом и загрузите его в Colab. Выполните задачи, сохраните изменения: воспользуйтесь опцией Save and Checkpoint из вкладки меню File или кнопкой Save and Checkpoint на панели инструментов. Отправьте через форму ниже итоговый файл Jupyter Notebook (.ipynb) или ссылку на него.